### Natural language media search
### Goals & scope
- Given one or many video/audio files and a natural-language query, return the most relevant segments with: start/end timestamps, transcript snippet, thumbnail(s), and a relevance score.
- **MVP scope** : Single-machine prototype that can index and search a handful of videos (hours of content).
- **Core capabilities** to learn: audio transcription and alignment, audio/visual/text embeddings, vector indexing (FAISS), multimodal retrieval & reranking.

### 🧩 Media_Preprocessor

We begin by **standardizing media input (video or audio)** into a consistent format suitable for downstream **NLP**, **ASR**, or **computer vision** models.  
This pipeline ensures every file is **normalized**, **well-structured**, and **frame-aligned** for reproducible processing.

---

#### 1. **Initialization**
When creating a `MediaPreprocessor` instance, several configurable parameters define the workflow:
- `keyframe_interval`: Interval (in seconds) for fallback frame extraction.
- `target_sample_rate`: The audio resampling rate (default 16 kHz for ASR models like Whisper).
- `frame_size`: Output resolution for extracted frames (default 224×224 for CV models).
- `target_lufs`: Loudness normalization target (−16 LUFS for broadcast-quality speech).
- `scene_detect`: Enables scene-based keyframe detection using ffmpeg’s `scene` filter.
- `scene_threshold`: Sensitivity for detecting visual changes between scenes.
- `verbose`: Prints ffmpeg progress logs if `True`.

---

#### 2. **Temporary Workspace**
Each processed file gets a unique working directory inside `./media_processed`, 
named as `<filename>_<uuid>`.  
This isolation prevents data collisions and simplifies debugging or cleanup.

---

#### 3. **Audio Extraction & Normalization**
We extract **mono 16 kHz PCM WAV** audio using ffmpeg.  
If `normalize=True`, the **loudnorm** filter aligns average loudness to the target LUFS, 
ensuring consistent volume levels across clips — crucial for speech recognition models.

---

#### 4. **Metadata Extraction**
Using `ffprobe`, we retrieve technical details such as:
- Duration
- Codec type & name
- Frame dimensions or sample rate
- Bitrate and format name  
All metadata is written to a JSON summary in the run directory.

---

#### 5. **Scene-Based Keyframe Extraction**
If `scene_detect=True`, ffmpeg analyzes visual changes in the video to identify **scene transitions**.  
Frames corresponding to these timestamps are extracted as key images.  
This yields semantically meaningful frames for visual embeddings or captioning.

If no scene changes are detected (or disabled), the system falls back to **fixed interval extraction**, 
grabbing one frame every *N* seconds (`keyframe_interval`).

---




In [61]:
# media_preprocessor.py
import ffmpeg
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import math
import uuid
import shutil
import os
import subprocess
import json
import time

try:
    import webrtcvad
    _VAD_AVAILABLE = True
except Exception:
    _VAD_AVAILABLE = False


class MediaPreprocessor:
    """
    A robust media preprocessing pipeline for both video and audio files.
    Handles:
        - Audio normalization to target LUFS
        - Frame extraction using scene-detection or fixed interval
        - Metadata extraction (duration, codec info, frame size)
        - Organized output in unique temp directories
    """

    def __init__(self,
                 keyframe_interval: int = 1,
                 target_sample_rate: int = 16000,
                 frame_size: Tuple[int, int] = (224, 224),
                 temp_dir: str = "./media_processed",
                 target_lufs: float = -16.0,
                 scene_detect: bool = True,
                 scene_threshold: float = 0.2,
                 verbose: bool = False,
                 keep_intermediates: bool = True):
        """
        Initialize preprocessing configuration.
        """
        self.keyframe_interval = int(keyframe_interval)
        self.target_sample_rate = int(target_sample_rate)
        self.frame_size = frame_size
        self.root_temp_dir = Path(temp_dir)
        self.root_temp_dir.mkdir(parents=True, exist_ok=True)
        self.target_lufs = float(target_lufs)
        self.scene_detect = bool(scene_detect)
        self.scene_threshold = float(scene_threshold)
        self.verbose = bool(verbose)
        self.keep_intermediates = bool(keep_intermediates)

        print("✅ MediaPreprocessor initialized:")
        print(f"  - Keyframe interval: {self.keyframe_interval}s")
        print(f"  - Target sample rate: {self.target_sample_rate}Hz")
        print(f"  - Frame size: {self.frame_size}")
        print(f"  - Root temp directory: {self.root_temp_dir.resolve()}")
        print(f"  - Target LUFS: {self.target_lufs}")
        print(f"  - Scene-detect: {self.scene_detect} (Threshold: {self.scene_threshold})")
        print(f"  - Verbose logging: {self.verbose}")
        print(f"  - Keep intermediates: {self.keep_intermediates}")

    # --------------------
    def _make_run_dir(self, file_path: Path) -> Path:
        """Create a unique temporary directory for this file."""
        uid = uuid.uuid4().hex[:8]
        out_dir = self.root_temp_dir / f"{file_path.stem}_{uid}"
        out_dir.mkdir(parents=True, exist_ok=True)
        return out_dir

    # --------------------
    def _get_duration(self, file_path: Path) -> float:
        """Get the duration (in seconds) of a media file using ffprobe."""
        try:
            probe = ffmpeg.probe(str(file_path))
            return float(probe["format"]["duration"])
        except ffmpeg.Error as e:
            raise RuntimeError(f"ffprobe failed for {file_path}: {e.stderr.decode() if e.stderr else e}")

    # --------------------
    def _extract_audio(self, file_path: Path, out_dir: Path, normalize: bool = True) -> Path:
        """Extract mono 16kHz WAV audio, optionally normalized."""
        out_path = out_dir / f"{file_path.stem}_audio.wav"
        stream = ffmpeg.input(str(file_path))

        if normalize:
            ffmpeg_stream = stream.audio.filter("loudnorm", I=self.target_lufs, TP=-1.5, LRA=7)
        else:
            ffmpeg_stream = stream.audio

        try:
            (
                ffmpeg_stream
                .output(str(out_path), format="wav", acodec="pcm_s16le", ac=1, ar=self.target_sample_rate)
                .overwrite_output()
                .run(quiet=(not self.verbose))
            )
        except ffmpeg.Error as e:
            raise RuntimeError(f"Audio extraction failed: {e.stderr.decode() if e.stderr else e}")
        return out_path

    # --------------------
    def _get_metadata(self, file_path: Path) -> Dict:
        """Retrieve video or audio metadata."""
        try:
            probe = ffmpeg.probe(str(file_path))
            fmt = probe["format"]
            streams = probe.get("streams", [])
            metadata = {
                "duration": float(fmt.get("duration", 0.0)),
                "bit_rate": fmt.get("bit_rate", "unknown"),
                "size": fmt.get("size", "unknown"),
                "format_name": fmt.get("format_name", "unknown"),
                "streams": []
            }
            for s in streams:
                entry = {k: s.get(k, None) for k in ["codec_type", "codec_name", "width", "height", "sample_rate"]}
                metadata["streams"].append(entry)
            return metadata
        except ffmpeg.Error as e:
            raise RuntimeError(f"ffprobe metadata retrieval failed: {e.stderr.decode() if e.stderr else e}")

    # --------------------
    def _get_scene_timestamps(self, file_path: Path) -> List[float]:
        """Detect scene changes using ffmpeg scene filter."""
        print(f"  Detecting scenes (threshold={self.scene_threshold})...")
        try:
            result = subprocess.run(
                [
                    "ffmpeg", "-i", str(file_path),
                    "-filter:v", f"select='gt(scene,{self.scene_threshold})',showinfo",
                    "-f", "null", "-"
                ],
                stderr=subprocess.PIPE, text=True, check=True
            )
            stderr = result.stderr
        except subprocess.CalledProcessError as e:
            stderr = e.stderr

        timestamps = []
        for line in stderr.splitlines():
            if "pts_time:" in line:
                try:
                    ts = float(line.split("pts_time:")[1].split(" ")[0])
                    timestamps.append(ts)
                except ValueError:
                    continue

        print(f"    Found {len(timestamps)} scene changes.")
        return sorted(set(timestamps))

    # --------------------
    def _extract_keyframes(self, file_path: Path, out_dir: Path) -> List[Dict]:
        """Extract keyframes using scene-detection or fixed interval."""
        frames = []
        if self.scene_detect:
            timestamps = self._get_scene_timestamps(file_path)
            if timestamps:
                for idx, ts in enumerate(timestamps):
                    out_path = out_dir / f"{file_path.stem}_scene_{idx:05d}_{ts:.3f}s.jpg"
                    try:
                        (
                            ffmpeg.input(str(file_path), ss=ts)
                            .filter("scale", self.frame_size[0], self.frame_size[1])
                            .output(str(out_path), vframes=1, q=2)
                            .overwrite_output()
                            .run(quiet=(not self.verbose))
                        )
                        frames.append({"path": str(out_path.resolve()), "timestamp": ts})
                    except ffmpeg.Error as e:
                        print(f"    Warning: failed to extract frame at {ts}s: {e}")
                print(f"    Extracted {len(frames)} scene-change frames.")
                if frames:
                    return frames

        # Fallback to fixed interval extraction
        print(f"  Using fixed interval keyframe extraction ({self.keyframe_interval}s)...")
        out_pattern = str(out_dir / f"{file_path.stem}_frame_%06d.jpg")
        fps_value = 1.0 / max(1, self.keyframe_interval)
        (
            ffmpeg.input(str(file_path))
            .filter("fps", fps=fps_value)
            .filter("scale", self.frame_size[0], self.frame_size[1])
            .output(out_pattern, vsync="vfr", format="image2", q=2)
            .overwrite_output()
            .run(quiet=(not self.verbose))
        )

        frame_files = sorted(out_dir.glob(f"{file_path.stem}_frame_*.jpg"))
        for idx, p in enumerate(frame_files):
            frames.append({"path": str(p.resolve()), "timestamp": float(idx * self.keyframe_interval)})

        print(f"    Extracted {len(frames)} fixed-interval frames.")
        return frames

    # --------------------
    def _process_video_file(self, file_path: Path) -> Dict:
        """Process a video file: extract audio + keyframes + metadata."""
        run_dir = self._make_run_dir(file_path)
        print(f"Processing video -> {run_dir.name}")
        audio_path = self._extract_audio(file_path, run_dir)
        keyframes = self._extract_keyframes(file_path, run_dir)
        metadata = self._get_metadata(file_path)

        data = {
            "file_type": "video",
            "original_file": str(file_path.resolve()),
            "working_dir": str(run_dir.resolve()),
            "main_audio_file": str(audio_path.resolve()),
            "video_keyframes": keyframes,
            "metadata": metadata
        }

        summary_path = run_dir / "metadata_summary.json"
        with open(summary_path, "w") as f:
            json.dump(data, f, indent=4)

        print(f"✅ Completed processing video: {file_path.name}")
        if not self.keep_intermediates:
            self.cleanup_run_dir(run_dir)
        return data

    # --------------------
    def _process_audio_file(self, file_path: Path) -> Dict:
        """Process an audio file: normalize and get metadata."""
        run_dir = self._make_run_dir(file_path)
        print(f"Processing audio -> {run_dir.name}")
        audio_path = self._extract_audio(file_path, run_dir)
        metadata = self._get_metadata(audio_path)

        data = {
            "file_type": "audio",
            "original_file": str(file_path.resolve()),
            "working_dir": str(run_dir.resolve()),
            "main_audio_file": str(audio_path.resolve()),
            "metadata": metadata
        }

        summary_path = run_dir / "metadata_summary.json"
        with open(summary_path, "w") as f:
            json.dump(data, f, indent=4)

        print(f"✅ Completed processing audio: {file_path.name}")
        if not self.keep_intermediates:
            self.cleanup_run_dir(run_dir)
        return data

    # --------------------
    def process_media_file(self, file_path: str) -> Dict:
        """Public entry point to process any supported video or audio file."""
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")

        video_exts = {'.mp4', '.mkv', '.mov', '.avi', '.webm', '.flv', '.wmv', '.mpeg', '.mpg', '.m4v', '.m2ts'}
        audio_exts = {'.mp3', '.wav', '.flac', '.aac', '.m4a', '.ogg', '.opus', '.wma', '.aiff'}

        ext = file_path.suffix.lower()
        if ext in video_exts:
            return self._process_video_file(file_path)
        elif ext in audio_exts:
            return self._process_audio_file(file_path)
        else:
            raise ValueError(f"Unsupported file format: {ext}")

    # --------------------
    def cleanup_run_dir(self, run_dir: Path):
        """Clean up temporary working directories."""
        if Path(run_dir).exists() and Path(run_dir).is_dir():
            shutil.rmtree(run_dir)
            print(f"🧹 Cleaned up {run_dir}")


In [62]:
import json


# your media 
my_media_file = "vid2.mp4"

# 2. Initialize the preprocessor
mp = MediaPreprocessor(scene_detect=True)

try:
    # 3. Call the method with your path
    result = mp.process_media_file(my_media_file)
    json_str= json.dumps(result, indent=2)
    print(json_str)
    with open("result.json", "w") as f:
      f.write(json_str)
except FileNotFoundError:
    print(f"Error: Could not find the file at: {my_media_file}")
except Exception as e:
    print(f"An error occurred: {e}")

In [63]:
processed_media_data = result

In [64]:
# === transcription_and_embedding.py ===
import os
import json
import numpy as np
import torch
import librosa
import faiss
from PIL import Image
from tqdm import tqdm
import whisper
from transformers import CLIPProcessor, CLIPModel


In [65]:
## Transcription
def transcribe_audio(processed_media_data, model_size="base"):
    """Transcribes the main audio file using Whisper."""
    if not processed_media_data or "main_audio_file" not in processed_media_data:
        raise ValueError("main_audio_file missing in processed_media_data")

    audio_path = processed_media_data["main_audio_file"]
    print(f"🎙️ Transcribing audio: {audio_path}")

    # Load Whisper model
    whisper_model = whisper.load_model(model_size)

    # Transcribe
    result = whisper_model.transcribe(audio_path, word_timestamps=True, verbose=True)
    if 

    # Store transcription results
    processed_media_data["transcription"] = result["text"]
    processed_media_data["transcription_segments"] = result["segments"]

    # Save to disk
    out_json = os.path.join(processed_media_data["working_dir"], "asr_output.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(processed_media_data, f, indent=2)
    print(f"✅ Transcription saved to {out_json}")

    return processed_media_data



In [72]:
processed_media_data = transcribe_audio(processed_media_data)


In [73]:
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm
import torch

def generate_keyframe_captions_verbose(processed_media_data, device=None, blip_model_name="Salesforce/blip-image-captioning-base"):
    """
    Generate BLIP captions for each keyframe and print detailed info for each step.

    Parameters:
        processed_media_data: dict from MediaPreprocessor containing 'video_keyframes'
        device: "cuda" or "cpu" (auto-detect if None)
        blip_model_name: BLIP model identifier

    Returns:
        captions: list of captions for each keyframe
        updated_media_data: same dict with 'caption' added to each keyframe
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    keyframes = processed_media_data.get("video_keyframes", [])
    if not keyframes:
        print("No keyframes found in media data.")
        return [], processed_media_data

    print(f"Loading BLIP model ({blip_model_name}) on {device}...")
    processor = BlipProcessor.from_pretrained(blip_model_name)
    model = BlipForConditionalGeneration.from_pretrained(blip_model_name).to(device)
    model.eval()

    captions = []

    print(f"Starting captioning for {len(keyframes)} keyframes...\n")
    for idx, frame in enumerate(keyframes, 1):
        path = frame.get("path", "unknown")
        ts = frame.get("timestamp", "N/A")
        print(f"[{idx}/{len(keyframes)}] Frame: {path}, Timestamp: {ts}")

        try:
            img = Image.open(path).convert("RGB")
            inputs = processor(images=img, return_tensors="pt").to(device)
            with torch.no_grad():
                out = model.generate(**inputs, max_new_tokens=40)
            caption = processor.decode(out[0], skip_special_tokens=True)
        except Exception as e:
            caption = ""
            print(f"  -> Error generating caption: {e}")

        frame["caption"] = caption
        captions.append(caption)
        print(f"  -> Generated Caption: {caption}\n")

    print("Captioning completed.\n")
    return captions, processed_media_data


In [76]:
generate_keyframe_captions_verbose(processed_media_data)

In [77]:
print(processed_media_data.get("transcription_segments",{}))

In [78]:
from IPython.display import display, Markdown

def build_scene_texts_with_preview(processed_media_data, fusion_window=5.0):
    """
    Build enriched text per keyframe/segment including:
     - frame caption (if any)
     - nearest transcription text around the timestamp
     - optional user metadata (title/characters)
    Displays each scene in Jupyter notebook.

    Returns:
        scene_texts: list of dicts with 'frame_path', 'timestamp', 'combined_text'
    """
    scene_texts = []
    trans_segments = processed_media_data.get("transcription_segments", [])
    keyframes = processed_media_data.get("video_keyframes", [])
    user_meta_data = processed_media_data.get("user_metadata", {})

    print(f"Processing {len(keyframes)} keyframes and {len(trans_segments)} transcription segments...\n")

    for frame in keyframes:
        ts = float(frame.get("timestamp", 0.0))
        nearby_texts = []

        # find transcription segments overlapping ±fusion_window/2
        for seg in trans_segments:
            s, e = seg.get("start", 0.0), seg.get("end", 0.0)
            if (s <= ts + fusion_window/2) and (e >= ts - fusion_window/2):
                nearby_texts.append(seg.get("text", "").strip())

        # fallback: pick nearest segment
        if not nearby_texts and trans_segments:
            dists = [abs(((seg.get("start",0)+seg.get("end",0))/2) - ts) for seg in trans_segments]
            idx = int(np.argmin(dists))
            nearby_texts = [trans_segments[idx].get("text", "").strip()]

        # user metadata
        user_meta = ""
        title = user_meta_data.get("title")
        chars = user_meta_data.get("characters")
        if title:
            user_meta += f"Title: {title}. "
        if chars:
            user_meta += f"Characters: {', '.join(chars)}. "

        caption = frame.get("caption", "")
        combined = " ".join([user_meta, caption, " ".join(nearby_texts)]).strip()
        if not combined:
            combined = caption or (nearby_texts[0] if nearby_texts else "")

        frame_text_entry = {
            "frame_path": frame.get("path"),
            "timestamp": ts,
            "combined_text": combined
        }
        scene_texts.append(frame_text_entry)

        # Display in notebook
        print(f"--- Frame timestamp: {ts} ---")
        print(f"Frame path: {frame.get('path')}")
        print(f"Caption: {caption}")
        print(f"Nearby transcription: {' '.join(nearby_texts)}")
        print(f"User metadata: {user_meta}")
        display(Markdown(f"**Combined text:** {combined}\n"))

    # Also add audio-only scenes
    for seg in trans_segments:
        s, e = seg.get("start",0.0), seg.get("end",0.0)
        mid = (s + e)/2.0
        scene_texts.append({
            "frame_path": None,
            "timestamp": mid,
            "combined_text": seg.get("text", "").strip()
        })

    # Sort by timestamp
    scene_texts = sorted(scene_texts, key=lambda x: x["timestamp"])
    print(f"\nTotal scenes (including audio-only): {len(scene_texts)}")
    return scene_texts


In [82]:
scene_texts =build_scene_texts_with_preview(processed_media_data)

Processing 349 keyframes and 414 transcription segments...

--- Frame timestamp: 6.433333 ---
Frame path: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid2_a6dad684/vid2_scene_00000_6.433s.jpg
Caption: a black background with a white line and a white line
Nearby transcription: Ginnyang!
User metadata: 


**Combined text:** a black background with a white line and a white line Ginnyang!


--- Frame timestamp: 11.433333 ---
Frame path: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid2_a6dad684/vid2_scene_00001_11.433s.jpg
Caption: a man standing in a living room looking at his cell
Nearby transcription: Hello.
User metadata: 


**Combined text:** a man standing in a living room looking at his cell Hello.


--- Frame timestamp: 18.066667 ---
Frame path: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid2_a6dad684/vid2_scene_00002_18.067s.jpg
Caption: a woman in a kitchen talking on a cell
Nearby transcription: Eric Bachman, this is your mom. And you are not my baby. Not now, Ginnyang.
User metadata: 


...

--- Frame timestamp: 929.433333 ---
Frame path: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid2_a6dad684/vid2_scene_00347_929.433s.jpg
Caption: a group of people walking through a subway station
Nearby transcription: Jin-yang!
User metadata: 


**Combined text:** a group of people walking through a subway station Jin-yang!


--- Frame timestamp: 931.033333 ---
Frame path: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid2_a6dad684/vid2_scene_00348_931.033s.jpg
Caption: a large building with a lot of cars on it
Nearby transcription: Jin-yang!
User metadata: 


**Combined text:** a large building with a lot of cars on it Jin-yang!



Total scenes (including audio-only): 763


In [83]:
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm
from typing import List, Dict, Tuple
from transformers import CLIPProcessor, CLIPModel
from transformers import CLIPProcessor, CLIPModel

# 1. Load pretrained CLIP model (e.g., ViT-B/32)
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

def l2_normalize(vec):
    return vec / np.linalg.norm(vec, axis=1, keepdims=True)

def embed_images(frames: List[Dict], model: CLIPModel, processor: CLIPProcessor, device="cuda" if torch.cuda.is_available() else "cpu", verbose=True) -> Tuple[np.ndarray, List[Dict]]:
    """
    Embed frames via CLIP image encoder.
    Returns:
        - np.ndarray of shape (N, D)
        - corresponding metadata list
    """
    model.to(device)
    model.eval()

    images_vecs = []
    meta = []

    for frame in tqdm(frames, desc="Embedding images", disable=not verbose):
        try:
            # Load image and preprocess
            img = Image.open(frame["path"]).convert("RGB")
            inputs = processor(images=img, return_tensors="pt").to(device)

            # Extract image features
            with torch.no_grad():
                feat = model.get_image_features(**inputs)  # [1, D]

            # Convert to NumPy and normalize
            vec = feat.cpu().numpy().astype("float32")
            vec = l2_normalize(vec)
            images_vecs.append(vec[0])

            # Metadata
            meta.append({
                "type": "image",
                "path": frame["path"],
                "timestamp": frame.get("timestamp")
            })

        except Exception as e:
            if verbose:
                print(f"Image embed error {frame.get('path')}: {e}")

    if images_vecs:
        return np.vstack(images_vecs), meta
    else:
        D = model.visual_projection.out_features
        return np.zeros((0, D), dtype="float32"), []


In [85]:
frames = [
    {"path": s["frame_path"], "timestamp": s["timestamp"]}
    for s in scene_texts if s["frame_path"] is not None
]
images_emb, images_meta = embed_images(frames, model, processor)


Embedding images: 100%|███████████████████████| 349/349 [00:11<00:00, 29.33it/s]


In [118]:
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
from PIL import Image
import base64, io
import plotly.express as px
from dash import Dash, dcc, html, Output, Input

def visualize_image_embeddings_with_text(embeddings, meta, perplexity=30):
    """
    Interactive TSNE scatter plot for image embeddings.
    Clicking a point shows:
    - the corresponding image
    - its combined text and timestamp
    """
    # 1️⃣ Reduce embeddings to 2D
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity)
    reduced = tsne.fit_transform(embeddings)

    df = pd.DataFrame({
        "x": reduced[:,0],
        "y": reduced[:,1],
        "path": [m["path"] for m in meta],
        "text": [m.get("combined_text", "") for m in meta],
        "timestamp": [m.get("timestamp") for m in meta]
    })

    # 2️⃣ Build Dash app
    app = Dash(__name__)

    app.layout = html.Div([
        html.Div([
            dcc.Graph(id='embedding-plot', figure=px.scatter(df, x="x", y="y", hover_data=["timestamp"]))
        ], style={'width': '65%', 'display': 'inline-block', 'vertical-align': 'top'}),

        html.Div([
            html.Img(id='clicked-image', src='', style={'max-width': '100%', 'max-height': '400px'}),
            html.Div(id='clicked-info', style={'margin-top': '10px'})
        ], style={'width': '32%', 'display': 'inline-block', 'padding-left': '20px'})
    ])

    # 3️⃣ Callback to update image + text on click
    @app.callback(
        Output('clicked-image', 'src'),
        Output('clicked-info', 'children'),
        Input('embedding-plot', 'clickData')
    )
    def display_clicked_image(clickData):
        if clickData is None:
            return '', ''
        point_idx = clickData['points'][0]['pointIndex']
        row = df.iloc[point_idx]

        # Convert image to base64
        img = Image.open(row['path']).convert("RGB")
        buf = io.BytesIO()
        img.save(buf, format="PNG")
        encoded = base64.b64encode(buf.getvalue()).decode()

        # Display combined text and timestamp
        info = [
            html.P(f"Timestamp: {row['timestamp']} s"),
            html.P(f"Text: {row['text']}")
        ]
        return f"data:image/png;base64,{encoded}", info

    # 4️⃣ Run the app
    app.run(debug=True)

# Usage
visualize_image_embeddings_with_text(images_emb, images_meta)


In [ ]:
def embed_texts(self, scene_texts: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """Embed combined textual descriptions using SentenceTransformer."""
        self._load_text_model()
        texts = [t["combined_text"] or "" for t in scene_texts]
        if not texts:
            return np.zeros((0, self._text_model.get_sentence_embedding_dimension()), dtype="float32"), []
        emb = self._text_model.encode(texts, show_progress_bar=self.verbose, convert_to_numpy=True, normalize_embeddings=True)
        meta = [{"type": "text", "timestamp": t["timestamp"], "frame_path": t["frame_path"], "text": t["combined_text"]} for t in scene_texts]
        return emb.astype("float32"), meta


In [120]:
import numpy as np
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

def embed_texts(scene_texts: List[Dict],
                model: SentenceTransformer = None,
                model_name: str = "all-MiniLM-L6-v2",
                verbose: bool = True) -> Tuple[np.ndarray, List[Dict]]:
    """
    Embed combined textual descriptions using SentenceTransformer.

    Args:
        scene_texts: List of dicts, each with 'combined_text', 'timestamp', 'frame_path'
        model: Preloaded SentenceTransformer model (optional)
        model_name: Model name to load if model not provided
        verbose: Whether to show progress bar

    Returns:
        embeddings: np.ndarray of shape (N, D)
        metadata: List of dicts corresponding to each embedding
    """
    # 1️⃣ Load model if not provided
    if model is None:
        model = SentenceTransformer(model_name)

    # 2️⃣ Extract texts
    texts = [t.get("combined_text", "") or "" for t in scene_texts]
    if not texts:
        return np.zeros((0, model.get_sentence_embedding_dimension()), dtype="float32"), []

    # 3️⃣ Compute embeddings
    emb = model.encode(texts,
                       show_progress_bar=verbose,
                       convert_to_numpy=True,
                       normalize_embeddings=True)

    # 4️⃣ Build metadata list
    meta = [
        {
            "type": "text",
            "timestamp": t.get("timestamp"),
            "frame_path": t.get("frame_path"),
            "text": t.get("combined_text")
        }
        for t in scene_texts
    ]

    return emb.astype("float32"), meta


In [121]:
# Import the function we created
from sentence_transformers import SentenceTransformer

# Load your text embedding model once
text_model = SentenceTransformer("all-MiniLM-L6-v2")  # or another SentenceTransformer model

# Embed scene texts
scene_texts_emb, scene_texts_meta = embed_texts(
    scene_texts=scene_texts,   # your scene_texts from build_scene_texts_with_preview
    model=text_model,          # pass the preloaded model
    verbose=True
)

# Check shapes and some metadata
print("Text embeddings shape:", scene_texts_emb.shape)
print("First scene text metadata:", scene_texts_meta[0])


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

Text embeddings shape: (763, 384)
First scene text metadata: {'type': 'text', 'timestamp': 0.64, 'frame_path': None, 'text': 'You smell something burning.'}


In [127]:
import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import umap

def visualize_scene_text_embeddings_interactive(embeddings: np.ndarray, meta: list,
                                                n_neighbors=15, min_dist=0.1, random_state=42):
    """
    Interactive 2D visualization of scene text embeddings.
    Clicking on a dot shows the full combined text on the side.
    """
    if embeddings.shape[0] == 0:
        print("No embeddings to visualize.")
        return
    
    # 1️⃣ Reduce to 2D using UMAP
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, metric='cosine', random_state=random_state)
    reduced = reducer.fit_transform(embeddings)
    
    # 2️⃣ Prepare DataFrame for Plotly
    df = pd.DataFrame({
        "x": reduced[:,0],
        "y": reduced[:,1],
        "combined_text": [m['text'] for m in meta],
        "timestamp": [m['timestamp'] for m in meta]
    })
    
    # Optional: clustering color by timestamp (or use other clustering algorithm)
    df['color'] = df['timestamp']

    # 3️⃣ Initialize Dash app
    app = Dash(__name__)

    fig = px.scatter(df, x="x", y="y", color="color", hover_data=["timestamp"], title="Scene Text Embeddings")
    
    app.layout = html.Div([
        html.Div([
            dcc.Graph(id="embedding-map", figure=fig)
        ], style={"width":"70%", "display":"inline-block", "verticalAlign":"top"}),

        html.Div([
            html.H4("Scene Text"),
            html.Div(id="text-display", style={"whiteSpace":"pre-wrap", "border":"1px solid #ccc",
                                              "padding":"10px", "height":"400px", "overflowY":"auto"})
        ], style={"width":"28%", "display":"inline-block", "marginLeft":"2%"})
    ])

    # 4️⃣ Update text display on click
    @app.callback(
        Output("text-display", "children"),
        Input("embedding-map", "clickData")
    )
    def display_text(clickData):
        if clickData is None:
            return "Click a dot to see scene text here."
        point_index = clickData['points'][0]['pointIndex']
        return df.iloc[point_index]['combined_text']

    # 5️⃣ Run app inline
    app.run(debug=True)


In [128]:
visualize_scene_text_embeddings_interactive(scene_texts_emb, scene_texts_meta)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [139]:
import numpy as np
import librosa
from PIL import Image
from typing import List, Dict, Tuple
import torch
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel
from scipy.signal import find_peaks

def l2_normalize(vec: np.ndarray) -> np.ndarray:
    return vec / (np.linalg.norm(vec, axis=1, keepdims=True) + 1e-12)

def embed_audio_emotional(audio_path: str,
                          transcription_segments: List[Dict],
                          clip_model: CLIPModel,
                          clip_processor: CLIPProcessor,
                          device="cuda" if torch.cuda.is_available() else "cpu",
                          sr=16000,
                          min_speech_duration=0.2,
                          peak_prominence=0.6,
                          energy_weight_scale=2.0,
                          verbose=True) -> Tuple[np.ndarray, List[Dict]]:
    """
    Embed audio segments emphasizing emotional/high-intensity events.
    
    Returns:
        - audio_embeddings: np.ndarray (N, D)
        - audio_meta: list of dicts with timestamps, text, type, and weight
    """
    clip_model.to(device)
    clip_model.eval()

    y, _ = librosa.load(audio_path, sr=sr, mono=True)
    audio_len = y.shape[0] / sr

    audio_vecs = []
    meta = []

    # 1️⃣ Embed ASR-aligned speech segments
    for seg in transcription_segments:
        start, end = seg.get("start", 0.0), seg.get("end", 0.0)
        if end <= start or (end - start) < min_speech_duration:
            continue
        seg_y = y[int(start*sr):int(end*sr)]

        # Mel spectrogram
        S = librosa.feature.melspectrogram(y=seg_y, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(S, ref=np.max)
        S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-12)
        S_img = Image.fromarray((S_norm*255).astype("uint8")).convert("RGB").resize((224,224))

        inputs = clip_processor(images=S_img, return_tensors="pt").to(device)
        with torch.no_grad():
            feat = clip_model.get_image_features(**inputs)
        vec = l2_normalize(feat.cpu().numpy())
        
        # Energy weighting based on RMS
        energy = float(np.mean(np.abs(seg_y)))
        weight = 1.0 + energy_weight_scale * energy  # more energetic segments get higher weight
        vec[0] *= weight

        audio_vecs.append(vec[0])
        meta.append({
            "type": "speech",
            "start_sec": start,
            "end_sec": end,
            "text": seg.get("text", ""),
            "energy": energy,
            "weight": weight
        })

    # 2️⃣ Detect high-energy non-speech events
    rms = librosa.feature.rms(y=y)[0]
    peaks, _ = find_peaks(rms, prominence=peak_prominence)
    peak_times = peaks / (len(rms)/audio_len)

    for t in peak_times:
        # skip if inside existing speech segments
        if any(s['start_sec'] <= t <= s['end_sec'] for s in meta if s['type']=="speech"):
            continue
        start = max(t - 0.25, 0.0)
        end = min(t + 0.25, audio_len)
        seg_y = y[int(start*sr):int(end*sr)]

        S = librosa.feature.melspectrogram(y=seg_y, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(S, ref=np.max)
        S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-12)
        S_img = Image.fromarray((S_norm*255).astype("uint8")).convert("RGB").resize((224,224))

        inputs = clip_processor(images=S_img, return_tensors="pt").to(device)
        with torch.no_grad():
            feat = clip_model.get_image_features(**inputs)
        vec = l2_normalize(feat.cpu().numpy())

        # Weight high-intensity non-speech segments more
        energy = float(np.mean(np.abs(seg_y)))
        weight = 1.0 + energy_weight_scale * energy
        vec[0] *= weight

        audio_vecs.append(vec[0])
        meta.append({
            "type": "non-speech",
            "start_sec": start,
            "end_sec": end,
            "text": None,
            "energy": energy,
            "weight": weight
        })

    # 3️⃣ Normalize vectors after weighting
    if audio_vecs:
        audio_vecs = np.vstack(audio_vecs)
        norms = np.linalg.norm(audio_vecs, axis=1, keepdims=True)
        audio_vecs = audio_vecs / (norms + 1e-12)
    else:
        D = clip_model.visual_projection.out_features
        audio_vecs = np.zeros((0, D), dtype="float32")

    if verbose:
        print(f"Embedded {len(audio_vecs)} audio segments (speech + high-intensity non-speech)")

    return audio_vecs.astype("float32"), meta


In [143]:
audio_path = processed_media_data['main_audio_file']
transcription_segments = processed_media_data["transcription_segments"]  # list of dicts with start/end/text
from transformers import CLIPProcessor, CLIPModel

model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name)
clip_processor = CLIPProcessor.from_pretrained(model_name)


# 2. Embed audio with emotional/high-intensity emphasis
audio_emb, audio_meta = embed_audio_emotional(
    audio_path=audio_path,
    transcription_segments=transcription_segments,
    clip_model=clip_model,
    clip_processor=clip_processor,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

Embedded 398 audio segments (speech + high-intensity non-speech)


In [144]:
import numpy as np
import pandas as pd
import umap
import plotly.express as px

def visualize_audio_embeddings(embeddings: np.ndarray, meta: list, perplexity=30, title="Audio Embedding Map"):
    """
    embeddings: np.ndarray of shape (N, D)
    meta: list of dicts with keys: 'text', 'start_sec', 'end_sec', 'emotion'
    """
    if embeddings.size == 0:
        print("No audio embeddings to visualize.")
        return

    # 1️⃣ Reduce dimensionality
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
    reduced = reducer.fit_transform(embeddings)

    # 2️⃣ Prepare DataFrame
    df = pd.DataFrame({
        "x": reduced[:,0],
        "y": reduced[:,1],
        "text": [m.get("text","") for m in meta],
        "start_sec": [m.get("start_sec",0.0) for m in meta],
        "end_sec": [m.get("end_sec",0.0) for m in meta],
        "emotion": [m.get("emotion","") for m in meta]
    })

    # 3️⃣ Interactive plot with hover
    fig = px.scatter(
        df, x="x", y="y", 
        color="emotion", 
        hover_data=["text","start_sec","end_sec"],
        title=title,
        width=900, height=600
    )

    fig.update_traces(marker=dict(size=10, opacity=0.7))
    fig.show()


In [145]:
# Assuming you have:
# audio_emb, audio_meta = embed_audio_emotional(...)
# Each meta dict now contains: "text", "start_sec", "end_sec", "emotion"

visualize_audio_embeddings(audio_emb, audio_meta)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [46]:
def build_fused_scene_vectors(self,
                                  images_emb: np.ndarray, images_meta: List[Dict],
                                  texts_emb: np.ndarray, texts_meta: List[Dict],
                                  audio_emb: np.ndarray, audio_meta: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """
        Align embeddings by timestamp into scenes (time windows) and fuse them using weighted average.
        Returns fused_vectors (N, D) and fused_metadata list
        """
        # Build a list of items with timestamp and vector (all standardized dims)
        items = []

        # We assume embeddings are L2-normalized already; ensure shapes
        D = None
        if images_emb.size:
            D = images_emb.shape[1]
            for i, m in enumerate(images_meta):
                items.append({"type":"image","timestamp":m["timestamp"], "vec":images_emb[i], "meta":m})
        if texts_emb.size:
            D = texts_emb.shape[1] if D is None else D
            for i, m in enumerate(texts_meta):
                items.append({"type":"text","timestamp":m["timestamp"], "vec":texts_emb[i], "meta":m})
        if audio_emb.size:
            D = audio_emb.shape[1] if D is None else D
            for i, m in enumerate(audio_meta):
                # audio metadata may not have a single timestamp; use mid-point if start/end present
                ts = None
                if m.get("start_sec") is not None and m.get("end_sec") is not None:
                    ts = (m["start_sec"] + m["end_sec"]) / 2.0
                else:
                    ts = m.get("start_sec", 0.0)
                items.append({"type":"audio","timestamp":ts,"vec":audio_emb[i],"meta":m})

        if not items:
            return np.zeros((0, D or 0), dtype="float32"), []

        # Sort items by timestamp
        items = sorted(items, key=lambda x: x["timestamp"])

        # Sliding window grouping: create scene centers spaced by fusion_window/2 over video duration
        duration = self.data.get("metadata", {}).get("duration")
        if duration is None:
            # try probing durations in metadata_summary or fallback to last timestamp
            duration = items[-1]["timestamp"] + self.fusion_window
        scene_centers = np.arange(0, max(duration, items[-1]["timestamp"]) + self.fusion_window, self.fusion_window)

        fused_vecs = []
        fused_meta = []
        for center in scene_centers:
            start = center - self.fusion_window/2.0
            end = center + self.fusion_window/2.0
            # collect vectors inside window
            image_vecs = [it["vec"] for it in items if it["type"]=="image" and (start <= it["timestamp"] <= end)]
            text_vecs  = [it["vec"] for it in items if it["type"]=="text"  and (start <= it["timestamp"] <= end)]
            audio_vecs = [it["vec"] for it in items if it["type"]=="audio" and (start <= it["timestamp"] <= end)]

            if not (image_vecs or text_vecs or audio_vecs):
                continue

            # average each modality (if exists)
            img_avg = np.mean(image_vecs, axis=0) if image_vecs else np.zeros((D,), dtype="float32")
            txt_avg = np.mean(text_vecs, axis=0) if text_vecs else np.zeros((D,), dtype="float32")
            aud_avg = np.mean(audio_vecs, axis=0) if audio_vecs else np.zeros((D,), dtype="float32")

            # fuse: weighted sum, then normalize
            fused = self.w_image * img_avg + self.w_text * txt_avg + self.w_audio * aud_avg
            fused = fused.astype("float32")
            fused = fused / (np.linalg.norm(fused) + 1e-12)

            # Collect metadata: what items contributed and center timestamp
            contributors = {
                "center_timestamp": center,
                "image_count": len(image_vecs),
                "text_count": len(text_vecs),
                "audio_count": len(audio_vecs)
            }
            fused_meta.append({
                "center_timestamp": center,
                "start": start,
                "end": end,
                "contributors": contributors
            })
            fused_vecs.append(fused)

        fused_arr = np.vstack(fused_vecs).astype("float32") if fused_vecs else np.zeros((0, D), dtype="float32")
        return fused_arr, fused_meta

In [47]:
def build_faiss_index(self, vectors: np.ndarray, ef_construction: int = 200, m: int = 32):
        """
        Builds and saves an HNSW index (cosine via L2-normalized vectors).
        HNSW uses neighborhood param 'm' and efConstruction.
        """
        if vectors.size == 0:
            raise ValueError("No vectors to index")

        d = vectors.shape[1]
        # ensure L2-normalized for cosine
        vectors = l2_normalize(vectors)

        # HNSW index (IndexHNSWFlat)
        index = faiss.IndexHNSWFlat(d, m)
        index.hnsw.efConstruction = ef_construction
        index.add(vectors)
        faiss.write_index(index, self.index_path)
        if self.verbose: print(f"FAISS HNSW index built and saved to {self.index_path} (N={vectors.shape[0]}, d={d})")
        self.faiss_index = index
        return index


In [48]:
def expand_query_variants(self, query: str) -> List[str]:
        """
        Simple query expansion: create several templated variants to improve recall.
        You can replace or augment with an LLM-based rewriter if available.
        """
        variants = [
            query,
            f"Scene where {query}",
            f"{query} scene",
            f"Someone is {query}",
            f"Action: {query}",
            f"Visual description: {query}"
        ]
        # dedupe
        seen = set()
        unique = []
        for v in variants:
            if v not in seen:
                unique.append(v)
                seen.add(v)
        return unique

In [49]:
# multimodal_indexer.py
import os
import json
import math
import numpy as np
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from PIL import Image
from tqdm import tqdm

import torch
import faiss
import librosa

# Transformers & models
import whisper
from transformers import CLIPProcessor, CLIPModel
from sentence_transformers import SentenceTransformer
from transformers import BlipProcessor, BlipForConditionalGeneration

# ---------------------------
# Utility helpers
# ---------------------------
def ensure_dir(p: str):
    Path(p).mkdir(parents=True, exist_ok=True)

def l2_normalize(a: np.ndarray) -> np.ndarray:
    """Normalize rows (or single vector) to unit L2 norm."""
    if a.ndim == 1:
        a = a / (np.linalg.norm(a) + 1e-12)
        return a
    norms = np.linalg.norm(a, axis=1, keepdims=True) + 1e-12
    return a / norms

# ---------------------------
# Multimodal Indexer
# ---------------------------
class MultimodalIndexer:
    """
    Builds multimodal embeddings and a FAISS index using:
      - CLIP for image embeddings (and audio spectrograms as images)
      - SentenceTransformer for text embeddings (transcripts + captions + metadata)
      - Whisper for transcription (if not present)
      - BLIP for image captions (NLP augmentation)
    Produces timestamp-aligned fused vectors and a FAISS HNSW index for searching.
    """
    def __init__(self,
                 processed_media_data: Dict[str, Any],
                 device: Optional[str] = None,
                 image_model_name: str = "openai/clip-vit-base-patch32",
                 text_model_name: str = "sentence-transformers/all-MiniLM-L6-v2",
                 blip_model_name: str = "Salesforce/blip-image-captioning-base",
                 whisper_model_size: str = "base",
                 fusion_window: float = 5.0,
                 fusion_weights: Tuple[float, float, float] = (0.4, 0.4, 0.2),
                 index_path: Optional[str] = None,
                 verbose: bool = True):
        """
        Parameters:
          processed_media_data: dict output from MediaPreprocessor
          device: "cuda" or "cpu" (auto-detected if None)
          fusion_window: time window in seconds to align items into one "scene"
          fusion_weights: (w_image, w_text, w_audio) weights when fusing
        """
        self.data = processed_media_data
        if self.data is None:
            raise ValueError("processed_media_data must be provided")

        # device auto
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        self.image_model_name = image_model_name
        self.text_model_name = text_model_name
        self.blip_model_name = blip_model_name
        self.whisper_model_size = whisper_model_size

        self.fusion_window = float(fusion_window)
        self.w_image, self.w_text, self.w_audio = fusion_weights

        self.index_path = index_path or os.path.join(self.data["working_dir"], "faiss_index.bin")
        self.metadata_path = os.path.join(self.data["working_dir"], "index_metadata.json")

        self.verbose = verbose

        # Models (lazy load)
        self._clip_model = None
        self._clip_processor = None
        self._text_model = None
        self._blip_processor = None
        self._blip_model = None
        self._whisper_model = None

        # Storage
        self.fused_vectors = None  # numpy (N, D)
        self.fused_metadata = []   # list of dicts
        self.faiss_index = None

    # --------------------
    # Lazy model loaders
    # --------------------
    def _load_clip(self):
        if self._clip_model is None:
            if self.verbose: print("Loading CLIP model...")
            self._clip_processor = CLIPProcessor.from_pretrained(self.image_model_name)
            self._clip_model = CLIPModel.from_pretrained(self.image_model_name).to(self.device)
            self._clip_model.eval()

    def _load_text_model(self):
        if self._text_model is None:
            if self.verbose: print("Loading text embedding model...")
            self._text_model = SentenceTransformer(self.text_model_name, device=self.device)

    def _load_blip(self):
        if self._blip_model is None:
            if self.verbose: print("Loading BLIP captioning model...")
            self._blip_processor = BlipProcessor.from_pretrained(self.blip_model_name)
            self._blip_model = BlipForConditionalGeneration.from_pretrained(self.blip_model_name).to(self.device)
            self._blip_model.eval()

    def _load_whisper(self):
        if self._whisper_model is None:
            if self.verbose: print("Loading Whisper model...")
            self._whisper_model = whisper.load_model(self.whisper_model_size)

    # --------------------
    # 2) NLP augmentation
    #    * image captions (BLIP)
    #    * local scene combined text (transcript + metadata + captions)
    # --------------------
    def generate_image_captions(self):
        """Generate captions for each keyframe using BLIP (if available)."""
        if "video_keyframes" not in self.data or not self.data["video_keyframes"]:
            if self.verbose: print("No video_keyframes found, skipping captions.")
            return []

        self._load_blip()
        captions = []
        for frame in tqdm(self.data["video_keyframes"], desc="Captioning keyframes", disable=not self.verbose):
            try:
                img = Image.open(frame["path"]).convert("RGB")
                inputs = self._blip_processor(images=img, return_tensors="pt").to(self.device)
                with torch.no_grad():
                    out = self._blip_model.generate(**inputs, max_new_tokens=40)
                caption = self._blip_processor.decode(out[0], skip_special_tokens=True)
            except Exception as e:
                caption = ""
                if self.verbose: print(f"Caption error for {frame['path']}: {e}")

            frame["caption"] = caption
            captions.append(caption)
        return captions

    def build_scene_texts(self):
        """
        Build an enriched text per keyframe/segment that includes:
         - frame caption (if any)
         - nearest transcription text around the timestamp
         - optional user metadata (title/characters) if provided in self.data
        """
        scene_texts = []
        trans_segments = self.data.get("transcription_segments", [])
        # Prebuild a fast lookup by timestamp for transcript segments
        # segments typically have 'start' and 'end' fields (Whisper)
        for frame in self.data.get("video_keyframes", []):
            ts = float(frame.get("timestamp", 0.0))
            # find transcription segments overlapping ±fusion_window/2
            window = self.fusion_window
            nearby_texts = []
            for seg in trans_segments:
                s = seg.get("start", 0.0)
                e = seg.get("end", 0.0)
                # overlap check
                if (s <= ts + window/2) and (e >= ts - window/2):
                    nearby_texts.append(seg.get("text", "").strip())

            # fallback: if no nearby transcripts, maybe pick nearest segment
            if not nearby_texts and trans_segments:
                # find nearest
                dists = [abs(((seg.get("start", 0)+seg.get("end",0))/2) - ts) for seg in trans_segments]
                idx = int(np.argmin(dists))
                nearby_texts = [trans_segments[idx].get("text", "").strip()]

            # user metadata (title, characters)
            user_meta = ""
            if "user_metadata" in self.data:
                # user can provide: {"title":"...", "characters":["A","B"], ...}
                um = self.data["user_metadata"]
                title = um.get("title")
                chars = um.get("characters")
                if title:
                    user_meta += f"Title: {title}. "
                if chars:
                    user_meta += f"Characters: {', '.join(chars)}. "

            caption = frame.get("caption", "")
            combined = " ".join([user_meta, caption, " ".join(nearby_texts)]).strip()
            if not combined:
                combined = caption or (nearby_texts[0] if nearby_texts else "")
            frame_text_entry = {
                "frame_path": frame["path"],
                "timestamp": ts,
                "combined_text": combined
            }
            scene_texts.append(frame_text_entry)

        # Also create pure scene entries from transcription segments alone (for audio-only scenes)
        for seg in self.data.get("transcription_segments", []):
            s = seg.get("start", 0.0)
            e = seg.get("end", 0.0)
            mid = (s + e) / 2.0
            scene_texts.append({
                "frame_path": None,
                "timestamp": mid,
                "combined_text": seg.get("text", "").strip()
            })

        # Sort by timestamp
        scene_texts = sorted(scene_texts, key=lambda x: x["timestamp"])
        return scene_texts

    # --------------------
    # 3) Embeddings per modality
    # --------------------
    def embed_images(self, frames: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """Embed frames via CLIP image encoder. Returns array (N, D) and matched metadata list."""
        self._load_clip()
        images_vecs = []
        meta = []
        for frame in tqdm(frames, desc="Embedding images", disable=not self.verbose):
            try:
                img = Image.open(frame["path"]).convert("RGB")
                inputs = self._clip_processor(images=img, return_tensors="pt").to(self.device)
                with torch.no_grad():
                    feat = self._clip_model.get_image_features(**inputs)
                vec = feat.cpu().numpy().astype("float32")
                vec = l2_normalize(vec)
                images_vecs.append(vec[0])
                meta.append({"type": "image", "path": frame["path"], "timestamp": frame["timestamp"]})
            except Exception as e:
                if self.verbose: print(f"Image embed error {frame['path']}: {e}")
        if images_vecs:
            return np.vstack(images_vecs), meta
        else:
            return np.zeros((0, self._clip_model.visual_projection.out_features), dtype="float32"), []

    def embed_texts(self, scene_texts: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """Embed combined textual descriptions using SentenceTransformer."""
        self._load_text_model()
        texts = [t["combined_text"] or "" for t in scene_texts]
        if not texts:
            return np.zeros((0, self._text_model.get_sentence_embedding_dimension()), dtype="float32"), []
        emb = self._text_model.encode(texts, show_progress_bar=self.verbose, convert_to_numpy=True, normalize_embeddings=True)
        meta = [{"type": "text", "timestamp": t["timestamp"], "frame_path": t["frame_path"], "text": t["combined_text"]} for t in scene_texts]
        return emb.astype("float32"), meta

    def embed_audio_segments(self, audio_segments: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """
        Convert audio segments to mel-spectrogram images and embed with CLIP image encoder.
        Expects each audio_segments item to have 'path','start_sec','end_sec' (or it can be whole file).
        """
        self._load_clip()
        audio_vecs = []
        meta = []
        for seg in tqdm(audio_segments, desc="Embedding audio", disable=not self.verbose):
            try:
                path = seg["path"]
                # Load audio (mono)
                y, sr = librosa.load(path, sr=16000)
                # Trim segment if start/end are given in seg (some workflows produce segment files already)
                start = seg.get("start_sec", 0.0)
                end = seg.get("end_sec", None)
                if end and end > start:
                    # Convert times to samples if needed; but if path is already segment file then just use whole y
                    # We assume path points to the segment; otherwise you'd need to load master audio and slice.
                    pass

                # Make mel spectrogram
                S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
                S_db = librosa.power_to_db(S, ref=np.max)
                S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min() + 1e-12)
                # Convert to 3-channel PIL image (resize to reasonable CLIP input if needed)
                S_img = Image.fromarray((S_norm * 255).astype("uint8")).convert("RGB").resize((224,224))

                inputs = self._clip_processor(images=S_img, return_tensors="pt").to(self.device)
                with torch.no_grad():
                    feat = self._clip_model.get_image_features(**inputs)
                vec = feat.cpu().numpy().astype("float32")
                vec = l2_normalize(vec)
                audio_vecs.append(vec[0])
                meta.append({"type": "audio", "path": path, "start_sec": seg.get("start_sec"), "end_sec": seg.get("end_sec")})
            except Exception as e:
                if self.verbose: print(f"Audio embed error {seg.get('path')}: {e}")

        if audio_vecs:
            return np.vstack(audio_vecs), meta
        else:
            return np.zeros((0, self._clip_model.visual_projection.out_features), dtype="float32"), []

    # --------------------
    # 4) Timestamp alignment & fusion
    # --------------------
    def build_fused_scene_vectors(self,
                                  images_emb: np.ndarray, images_meta: List[Dict],
                                  texts_emb: np.ndarray, texts_meta: List[Dict],
                                  audio_emb: np.ndarray, audio_meta: List[Dict]) -> Tuple[np.ndarray, List[Dict]]:
        """
        Align embeddings by timestamp into scenes (time windows) and fuse them using weighted average.
        Returns fused_vectors (N, D) and fused_metadata list
        """
        # Build a list of items with timestamp and vector (all standardized dims)
        items = []

        # We assume embeddings are L2-normalized already; ensure shapes
        D = None
        if images_emb.size:
            D = images_emb.shape[1]
            for i, m in enumerate(images_meta):
                items.append({"type":"image","timestamp":m["timestamp"], "vec":images_emb[i], "meta":m})
        if texts_emb.size:
            D = texts_emb.shape[1] if D is None else D
            for i, m in enumerate(texts_meta):
                items.append({"type":"text","timestamp":m["timestamp"], "vec":texts_emb[i], "meta":m})
        if audio_emb.size:
            D = audio_emb.shape[1] if D is None else D
            for i, m in enumerate(audio_meta):
                # audio metadata may not have a single timestamp; use mid-point if start/end present
                ts = None
                if m.get("start_sec") is not None and m.get("end_sec") is not None:
                    ts = (m["start_sec"] + m["end_sec"]) / 2.0
                else:
                    ts = m.get("start_sec", 0.0)
                items.append({"type":"audio","timestamp":ts,"vec":audio_emb[i],"meta":m})

        if not items:
            return np.zeros((0, D or 0), dtype="float32"), []

        # Sort items by timestamp
        items = sorted(items, key=lambda x: x["timestamp"])

        # Sliding window grouping: create scene centers spaced by fusion_window/2 over video duration
        duration = self.data.get("metadata", {}).get("duration")
        if duration is None:
            # try probing durations in metadata_summary or fallback to last timestamp
            duration = items[-1]["timestamp"] + self.fusion_window
        scene_centers = np.arange(0, max(duration, items[-1]["timestamp"]) + self.fusion_window, self.fusion_window)

        fused_vecs = []
        fused_meta = []
        for center in scene_centers:
            start = center - self.fusion_window/2.0
            end = center + self.fusion_window/2.0
            # collect vectors inside window
            image_vecs = [it["vec"] for it in items if it["type"]=="image" and (start <= it["timestamp"] <= end)]
            text_vecs  = [it["vec"] for it in items if it["type"]=="text"  and (start <= it["timestamp"] <= end)]
            audio_vecs = [it["vec"] for it in items if it["type"]=="audio" and (start <= it["timestamp"] <= end)]

            if not (image_vecs or text_vecs or audio_vecs):
                continue

            # average each modality (if exists)
            img_avg = np.mean(image_vecs, axis=0) if image_vecs else np.zeros((D,), dtype="float32")
            txt_avg = np.mean(text_vecs, axis=0) if text_vecs else np.zeros((D,), dtype="float32")
            aud_avg = np.mean(audio_vecs, axis=0) if audio_vecs else np.zeros((D,), dtype="float32")

            # fuse: weighted sum, then normalize
            fused = self.w_image * img_avg + self.w_text * txt_avg + self.w_audio * aud_avg
            fused = fused.astype("float32")
            fused = fused / (np.linalg.norm(fused) + 1e-12)

            # Collect metadata: what items contributed and center timestamp
            contributors = {
                "center_timestamp": center,
                "image_count": len(image_vecs),
                "text_count": len(text_vecs),
                "audio_count": len(audio_vecs)
            }
            fused_meta.append({
                "center_timestamp": center,
                "start": start,
                "end": end,
                "contributors": contributors
            })
            fused_vecs.append(fused)

        fused_arr = np.vstack(fused_vecs).astype("float32") if fused_vecs else np.zeros((0, D), dtype="float32")
        return fused_arr, fused_meta

    # --------------------
    # 5) Build FAISS index
    # --------------------
    def build_faiss_index(self, vectors: np.ndarray, ef_construction: int = 200, m: int = 32):
        """
        Builds and saves an HNSW index (cosine via L2-normalized vectors).
        HNSW uses neighborhood param 'm' and efConstruction.
        """
        if vectors.size == 0:
            raise ValueError("No vectors to index")

        d = vectors.shape[1]
        # ensure L2-normalized for cosine
        vectors = l2_normalize(vectors)

        # HNSW index (IndexHNSWFlat)
        index = faiss.IndexHNSWFlat(d, m)
        index.hnsw.efConstruction = ef_construction
        index.add(vectors)
        faiss.write_index(index, self.index_path)
        if self.verbose: print(f"FAISS HNSW index built and saved to {self.index_path} (N={vectors.shape[0]}, d={d})")
        self.faiss_index = index
        return index

    # --------------------
    # 6) Query expansion (simple template augmentation)
    # --------------------
    def expand_query_variants(self, query: str) -> List[str]:
        """
        Simple query expansion: create several templated variants to improve recall.
        You can replace or augment with an LLM-based rewriter if available.
        """
        variants = [
            query,
            f"Scene where {query}",
            f"{query} scene",
            f"Someone is {query}",
            f"Action: {query}",
            f"Visual description: {query}"
        ]
        # dedupe
        seen = set()
        unique = []
        for v in variants:
            if v not in seen:
                unique.append(v)
                seen.add(v)
        return unique

    # --------------------
    # 7) Public build pipeline
    # --------------------
    def build(self,
              do_transcribe: bool = True,
              do_caption: bool = True,
              build_index: bool = True,
              audio_segments: Optional[List[Dict]] = None):
        """
        Full pipeline orchestration:
          - ensure transcription
          - generate captions (if requested)
          - build enriched scene texts
          - embed images, texts, audio
          - fuse by timestamp and build FAISS index
        """
        ensure_dir(self.data["working_dir"])

        if do_transcribe:
            self.ensure_transcription()

        if do_caption:
            # caption keyframes (adds 'caption' into each keyframe dict)
            try:
                self.generate_image_captions()
            except Exception as e:
                if self.verbose: print(f"Caption generation failed: {e}. Proceeding without captions.")

        # build enriched scene texts (per-frame/segment combined text)
        scene_texts = self.build_scene_texts()

        # embed modalities
        images = self.data.get("video_keyframes", [])
        images_emb, images_meta = (np.zeros((0,512), dtype="float32"), []) if not images else self.embed_images(images)
        texts_emb, texts_meta = (np.zeros((0,self._text_model.get_sentence_embedding_dimension()), dtype="float32"), []) if not scene_texts else self.embed_texts(scene_texts)

        # audio segments: either provided as pre-sliced segments in processed_media_data["audio_segments"] or argument
        audio_segs = audio_segments or self.data.get("audio_segments", [])
        audio_emb, audio_meta = (np.zeros((0, images_emb.shape[1] if images_emb.size else 512), dtype="float32"), []) if not audio_segs else self.embed_audio_segments(audio_segs)

        # if text dim != image dim, project text into image space or vice versa.
        # For simplicity, if dims differ, project text into image dim via a linear projection trained by PCA or random init.
        # Here we check dims and make a simple PCA projection if needed (quick solution).
        # But preferred approach: train a small linear mapping if you have paired data.
        if texts_emb.size and images_emb.size and texts_emb.shape[1] != images_emb.shape[1]:
            # Quick proj: PCA on concatenation (not ideal but workable for prototyping)
            from sklearn.decomposition import PCA
            target_dim = images_emb.shape[1]
            concat = np.vstack([texts_emb, images_emb])
            pca = PCA(n_components=min(target_dim, concat.shape[1]-1))
            proj = pca.fit_transform(concat)  # this yields combined projected features
            # split back
            texts_emb_proj = proj[:texts_emb.shape[0], :]
            images_emb = proj[texts_emb.shape[0]:, :]  # replace images with projected combined (approx)
            texts_emb = texts_emb_proj
            # normalize
            texts_emb = l2_normalize(texts_emb)
            images_emb = l2_normalize(images_emb)

        # Fuse into scene-level vectors
        fused_vectors, fused_meta = self.build_fused_scene_vectors(images_emb, images_meta, texts_emb, texts_meta, audio_emb, audio_meta)
        if fused_vectors.size == 0:
            raise ValueError("No fused scene vectors produced – nothing to index.")

        self.fused_vectors = fused_vectors
        self.fused_metadata = fused_meta

        if build_index:
            self.build_faiss_index(fused_vectors)

        # Save metadata mapping
        with open(self.metadata_path, "w", encoding="utf-8") as f:
            json.dump(self.fused_metadata, f, indent=2)
        if self.verbose: print("Metadata saved:", self.metadata_path)

        return self.faiss_index, self.fused_vectors, self.fused_metadata

    # --------------------
    # 8) Search function
    # --------------------
    def search_video(self, text_query: str, k: int = 5) -> List[Dict]:
        """
        Search the FAISS index using a text query.
        Returns top-k fused scenes with metadata and distances.
        """
        if self.faiss_index is None:
            # try load
            if os.path.exists(self.index_path):
                self.faiss_index = faiss.read_index(self.index_path)
            else:
                raise ValueError("FAISS index not built yet. Run build() first.")

        # Expand query variants and embed them (average their vectors for robust query)
        self._load_text_model()
        variants = self.expand_query_variants(text_query)
        embeddings = self._text_model.encode(variants, convert_to_numpy=True, normalize_embeddings=True)
        query_vec = np.mean(embeddings, axis=0).astype("float32")
        query_vec = l2_normalize(query_vec).reshape(1, -1)

        # if fused vector dim != query dim, attempt adaptation
        if query_vec.shape[1] != self.fused_vectors.shape[1]:
            # if dims mismatch, we fallback to projecting via PCA fit on fused vectors
            from sklearn.decomposition import PCA
            target_dim = self.fused_vectors.shape[1]
            if self.verbose: print("Query dim mismatch, projecting query to index dim.")
            pca = PCA(n_components=target_dim)
            _ = pca.fit(self.fused_vectors)  # fit PCA on fused vectors
            qproj = pca.transform(query_vec)
            qvec = l2_normalize(qproj).astype("float32")
        else:
            qvec = query_vec

        # Search
        D, I = self.faiss_index.search(qvec, k)
        results = []
        for rank in range(len(I[0])):
            idx = int(I[0][rank])
            dist = float(D[0][rank])
            meta = self.fused_metadata[idx] if idx < len(self.fused_metadata) else {}
            results.append({
                "rank": rank + 1,
                "index_id": idx,
                "distance": dist,
                "metadata": meta
            })
        return results

    # --------------------
    # Save/load utilities
    # --------------------
    def save_fused_vectors(self, path: Optional[str] = None):
        p = path or os.path.join(self.data["working_dir"], "fused_vectors.npy")
        np.save(p, self.fused_vectors)
        if self.verbose: print("Saved fused vectors to", p)
        return p

    def load_index_and_metadata(self, index_path: Optional[str] = None, metadata_path: Optional[str] = None):
        idx_p = index_path or self.index_path
        md_p = metadata_path or self.metadata_path
        if os.path.exists(idx_p):
            self.faiss_index = faiss.read_index(idx_p)
        if os.path.exists(md_p):
            with open(md_p, "r", encoding="utf-8") as f:
                self.fused_metadata = json.load(f)
        if self.verbose: print("Loaded index and metadata.")



In [50]:
myindexer = MultimodalIndexer(processed_media_data)
    

In [51]:

indexer = MultimodalIndexer(processed_media_data, verbose=True)
faiss_index, fused_vectors, fused_metadata = indexer.build(
    do_transcribe=False,  # already done
    do_caption=True,
    build_index=True
)

# Save metadata & vectors
with open(os.path.join(processed_media_data["working_dir"], "fused_metadata.json"), "w") as f:
    json.dump(fused_metadata, f, indent=2)
print("✅ Multimodal index built and metadata saved.")



Loading BLIP captioning model...


Captioning keyframes: 100%|███████████████████| 349/349 [02:22<00:00,  2.46it/s]


Loading CLIP model...


Embedding images: 100%|███████████████████████| 349/349 [00:13<00:00, 26.41it/s]


Loading text embedding model...


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 384 and the array at index 1 has size 512

In [ ]:
query = "child playing in park"
results = indexer.search_video(query, k=5)
print("Top search results:")
print(json.dumps(results, indent=2))

In [244]:
import json
import concurrent.futures # Keep import in case we want to switch back easily
import os

import whisper
# Proceed only if processed_media_data is available AND whisper is imported and valid
if ('processed_media_data' in globals() and processed_media_data is not None ):
    print("✅ 'processed_media_data' is defined and Whisper is available. Proceeding with transcription.")

    # 1️⃣ Load the Whisper model
    # Use the loaded model from a previous cell if available, otherwise load it
    # Use the imported whisper name
    if 'whispermodel' not in globals() or model is None:
         whisper_model = whisper.load_model("base")  # you can also use "small", "medium", or "large"
         print("✅ Whisper model loaded.")
    else:
         print("✅ Using existing Whisper model.")



✅ 'processed_media_data' is defined and Whisper is available. Proceeding with transcription.
✅ Whisper model loaded.


In [250]:
#Check for the new "main_audio_file" key
if processed_media_data and "main_audio_file" in processed_media_data:
    print("✅ Preprocessing data found. Proceeding with transcription.")

    # 1️⃣ Load the Whisper model
    if 'whisper_model' not in globals() or whisper_model is None:
        whisper_model = whisper.load_model("base")
        print("✅ Whisper model loaded.")
    else:
        print("✅ Using existing Whisper model.")

    main_audio_path = processed_media_data["main_audio_file"]
    print(f"🎙️ Transcribing 1-hour file: {main_audio_path}")
    print("   (This will take a while, but it is the correct way...)")

    try:
        # 2️⃣ Transcribe the ENTIRE file at once.
        # This is the fix. No more "for loop".
        result = whisper_model.transcribe(main_audio_path, 
                                          word_timestamps=True, 
                                          verbose=True) # Set to True to see progress

        print("\n✅ Transcription complete!")

        # 3️⃣ Add the full transcription results back to your data
        processed_media_data["transcription"] = result["text"]
        
        # This gets all of Whisper's internal segments
        processed_media_data["transcription_segments"] = result["segments"] 
        
        # 4️⃣ Save all results
        output_json_path = processed_media_data["working_dir"] + "/asr_output.json"
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(processed_media_data, f, indent=2)
        print(f"✅ Full transcription results saved to: {output_json_path}")

    except Exception as e:
        print(f"Error during transcription: {e}")

else:
    if processed_media_data:
        print("main_audio_file not found")
   

✅ Preprocessing data found. Proceeding with transcription.
✅ Using existing Whisper model.
🎙️ Transcribing 1-hour file: /Users/pratyushkhanal/Desktop/seniorseminar/media_nlp/media_processed/vid3_395a9d98/vid3_audio.wav
   (This will take a while, but it is the correct way...)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:09.020 --> 00:13.740]  Good evening welcome to the Spanish Super Cup second leg.
[00:15.160 --> 00:16.760]  It's awesome, it's really good.
[00:17.180 --> 00:23.080]  I think we've seen this one before but every time you always expect the best and we've
[00:23.080 --> 00:26.580]  had some great clashes between these two clubs.
[00:26.580 --> 00:30.760]  In 2011 and over the years.
[00:31.240 --> 00:37.700]  The last time Real Madrid came to the new camp they went away not too happy, a five-nil
[00:37.700 --> 00:43.860]  drop in last November in the league and of course they was the Champions League but I think
[00:43.860 --> 00:48.780]  Barcelona fans would rather remember the last league visit it really was.
[00:49.720 --> 00:56.120]  A tone setter for the rest of the season it marked a difference, a great big difference
[00:56.120 --> 00:57.260]  between these 

In [251]:
# Check if 'processed_media_data' is defined and has audio segments
if 'processed_media_data' in globals() and processed_media_data is not None and "audio_segments" in processed_media_data:
    print("--- Transcriptions ---")
    # Iterate through audio segments and print the transcription
    for i, segment in enumerate(processed_media_data["audio_segments"]):
        transcription = segment.get("transcription", "No transcription available")
        start_time = segment.get("start_sec", "N/A")
        end_time = segment.get("end_sec", "N/A")
        print(f"Segment {i+1} ({start_time:.2f}s - {end_time:.2f}s): {transcription}")
    print("--- End of Transcriptions ---")
else:
    print("'processed_media_data' is not defined or does not contain audio segments. Please run the preprocessing and transcription steps first.")

'processed_media_data' is not defined or does not contain audio segments. Please run the preprocessing and transcription steps first.


Sentence Transformers to convert the transcript snippets into dense vector representations suitable for similarity search against a natural language query.

In [252]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import librosa
import numpy as np
import faiss
import json

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load the pre-trained CLIP model
model_name = "openai/clip-vit-large-patch14"
clip_model = CLIPModel.from_pretrained(model_name).to(device)
clip_processor = CLIPProcessor.from_pretrained(model_name)

Using device: cpu


In [253]:
all_vectors = []
all_metadata = []

# Get the embedding dimension from the model config
embed_dim = clip_model.config.text_config.hidden_size #

# --- 1. Embed Text Transcriptions ---
print("Embedding text transcriptions...")
for segment in processed_media_data["audio_segments"]:
    transcription = segment.get("transcription", "")
    if not transcription or transcription.startswith("Error:"):
        continue

    # Process and embed the text
    inputs = processor(text=transcription, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    
    # Normalize and store
    text_vector = text_features.cpu().numpy().astype('float32')
    faiss.normalize_L2(text_vector) # Normalize for FAISS L2 search
    all_vectors.append(text_vector)
    all_metadata.append({
        "type": "text",
        "content": transcription,
        "start_sec": segment["start_sec"],
        "end_sec": segment["end_sec"]
    })

# --- 2. Embed Video Keyframes ---
print("Embedding video keyframes...")
for frame in processed_media_data["video_keyframes"]:
    try:
        image = Image.open(frame["path"])
        
        # Process and embed the image
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            image_features = clip_model.get_image_features(**inputs)
        
        # Normalize and store
        image_vector = image_features.cpu().numpy().astype('float32')
        faiss.normalize_L2(image_vector)
        all_vectors.append(image_vector)
        all_metadata.append({
            "type": "image",
            "path": frame["path"],
            "timestamp": frame["timestamp"]
        })
    except Exception as e:
        print(f"Error processing image {frame['path']}: {e}")

# --- 3. Embed Audio Segments (as Spectrograms) ---
print("Embedding audio spectrograms...")
for segment in processed_media_data["audio_segments"]:
    try:
        # Load audio file
        y, sr = librosa.load(segment["path"], sr=16000) # Use sample rate from metadata if needed
        
        # Create a Mel spectrogram
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(S, ref=np.max)
        
        # Normalize and convert to 3-channel PIL Image (to mimic RGB)
        S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min())
        S_img_array = (S_norm * 255).astype(np.uint8)
        S_pil = Image.fromarray(S_img_array).convert("RGB")
        
        # Process and embed the spectrogram *as an image*
        inputs = processor(images=S_pil, return_tensors="pt").to(device)
        with torch.no_grad():
            audio_features = clip_model.get_image_features(**inputs)
        
        # Normalize and store
        audio_vector = audio_features.cpu().numpy().astype('float32')
        faiss.normalize_L2(audio_vector)
        all_vectors.append(audio_vector)
        all_metadata.append({
            "type": "audio",
            "path": segment["path"],
            "start_sec": segment["start_sec"],
            "end_sec": segment["end_sec"]
        })
    except Exception as e:
        print(f"Error processing audio {segment['path']}: {e}")

print(f"Total assets indexed: {len(all_vectors)}")

# Stack all vectors into a single NumPy array
index_vectors = np.vstack(all_vectors)

Embedding text transcriptions...


KeyError: 'audio_segments'

In [220]:
# Create a FAISS index
# IndexFlatL2 uses L2 distance (Euclidean)
# Since we normalized our vectors, L2 distance is equivalent to cosine similarity, which is what CLIP uses.
index = faiss.IndexFlatL2(embed_dim)

# Add all our vectors to the index
index.add(index_vectors)

print(f"FAISS index built with {index.ntotal} vectors.")

# You can save the index and metadata for later
# faiss.write_index(index, "my_video.index")
# with open("my_video_metadata.json", "w") as f:
#    json.dump(all_metadata, f)

FAISS index built with 2617 vectors.


In [221]:
import os
import base64
from IPython.display import display, HTML, Video

# --- Make sure all your previous imports are loaded ---
# import torch
# from transformers import CLIPProcessor, CLIPModel
# from PIL import Image
# import librosa
# import numpy as np
# import faiss
# import json

In [222]:
def search_video(text_query, k=5):
    """
    Embeds a text query, searches the FAISS index, and returns the top k results.
    """
    print(f"\nSearching for: '{text_query}'")
    
    # 1. Embed the text query
    inputs = clip_processor(text=text_query, return_tensors="pt").to(device)
    with torch.no_grad():
        query_features = clip_model.get_text_features(**inputs)
    
    # 2. Normalize the query vector
    query_vector = query_features.cpu().numpy().astype('float32')
    faiss.normalize_L2(query_vector)
        
    # 3. Search the FAISS index
    D, I = index.search(query_vector, k)
    
    # 4. Package up the results
    results = []
    print("Top results found. Generating clips...")
    for i in range(k):
        result_index = I[0][i]
        result_metadata = all_metadata[result_index]
        distance = D[0][i]
        results.append({
            "rank": i + 1,
            "metadata": result_metadata,
            "distance": distance
        })
        
    return results



In [223]:
def display_search_results_with_clips(results, original_video_path, working_dir, clip_duration=5):
    """
    Generates and displays search results with embedded video clips.
    
    Assumes 'ffmpeg' is installed and accessible in your system's PATH.
    """
    
    # We will build a single HTML string to display all results
    html_output = "<div>"
    
    for res in results:
        metadata = res["metadata"]
        distance = res["distance"]
        rank = res["rank"]
        
        # --- 1. Determine Timeframe & Content ---
        content_info = ""
        if metadata["type"] == "text" or metadata["type"] == "audio":
            start_sec = metadata["start_sec"]
            duration = metadata["end_sec"] - metadata["start_sec"]
            content_info = f"<b>[{metadata['type'].upper()}]</b> at {start_sec:.2f}s"
            if metadata['type'] == 'text':
                content_info += f": '<i>{metadata['content']}</i>'"
        
        elif metadata["type"] == "image":
            # For an image, create a clip centered on the timestamp
            start_sec = max(0, metadata["timestamp"] - (clip_duration / 2))
            duration = clip_duration
            content_info = f"<b>[IMAGE]</b> at {metadata['timestamp']:.2f}s (Source frame: {metadata['path']})"
        
        # --- 2. Generate the Clip using FFmpeg ---
        clip_filename = f"search_result_rank_{rank}.mp4"
        clip_output_path = os.path.join(working_dir, clip_filename)
        
        # FFmpeg command:
        # -y: Overwrite output file
        # -ss: Seek to start time
        # -i: Input file
        # -t: Duration of the clip
        # -c:v libx264: Re-encode video (safer for clips)
        # -preset ultrafast: Encode very quickly
        # -c:a aac: Re-encode audio
        # -vf "scale=480:-1": Resize to 480px width, maintain aspect ratio
        ffmpeg_command = (
            f"ffmpeg -y -ss {start_sec} -i \"{original_video_path}\" "
            f"-t {duration} -c:v libx264 -preset ultrafast -c:a aac -vf \"scale=480:-1\" "
            f"\"{clip_output_path}\""
        )
        
        video_html = ""
        try:
            # Run the command (silence output with >/dev/null 2>&1)
            os.system(f"{ffmpeg_command} >/dev/null 2>&1")
            
            # --- 3. Embed Video using Base64 ---
            # This is robust and works well in all notebooks
            with open(clip_output_path, "rb") as f:
                video_data = f.read()
            video_base64 = base64.b64encode(video_data).decode("utf-8")
            video_src = f"data:video/mp4;base64,{video_base64}"
            
            video_html = f'<video controls width="480" src="{video_src}" type="video/mp4">Your browser does not support the video tag.</video>'
            
        except Exception as e:
            video_html = f"<p>Error generating video clip: {e}</p>"

        # --- 4. Build HTML for this result ---
        html_output += f"""
        <div style="border: 1px solid #ccc; border-radius: 8px; padding: 16px; margin-bottom: 16px; display: flex; align-items: top; flex-wrap: wrap; background-color: #f9f9f9;">
            <div style="flex: 1; min-width: 300px; padding-right: 16px;">
                <h3 style="margin-top:0;">Rank {rank} (Distance: {distance:.4f})</h3>
                <p>{content_info}</p>
            </div>
            <div style="flex-shrink: 0;">
                {video_html}
            </div>
        </div>
        """

    html_output += "</div>"
    
    # Display the final combined HTML
    display(HTML(html_output))

In [228]:
# --- Load your processed_media_data first ---
# (Assuming 'processed_media_data' is loaded from your JSON file)
# with open("path/to/your/asr_output.json", "r") as f:
#     processed_media_data = json.load(f)

# Get the two paths we need from your main data object
original_video_path = processed_media_data["original_file"]
working_dir = processed_media_data["working_dir"]

# --- 1. Run the search ---
query = "manager on frame"
search_results = search_video(query, k=5)
# --- Try it out! ---

# --- 2. Display results with clips ---
display_search_results_with_clips(search_results, original_video_path, working_dir)


Searching for: 'manager on frame'
Top results found. Generating clips...


In [209]:
search_video(query, k=5)


Searching for: 'screaming'
Top results found. Generating clips...


[{'rank': 1,
  'metadata': {'type': 'text',
   'content': " and I'm dead.",
   'start_sec': 347.96999999999997,
   'end_sec': 348.84},
  'distance': 0.11975021},
 {'rank': 2,
  'metadata': {'type': 'text',
   'content': ' gotta kill me.',
   'start_sec': 865.17,
   'end_sec': 865.98},
  'distance': 0.14849968},
 {'rank': 3,
  'metadata': {'type': 'text',
   'content': ' and thus.',
   'start_sec': 427.2,
   'end_sec': 427.83},
  'distance': 0.1565324},
 {'rank': 4,
  'metadata': {'type': 'text',
   'content': ' this.',
   'start_sec': 864.3299999999999,
   'end_sec': 864.8399999999999},
  'distance': 0.16108495},
 {'rank': 5,
  'metadata': {'type': 'text',
   'content': ' Oh, here.',
   'start_sec': 711.2099999999999,
   'end_sec': 711.72},
  'distance': 0.16588596}]

In [277]:
try:
    model = whisper.load_model("small")
    transcript = None
except Exception as e:
    print('no modelo')

In [283]:
import whisper
import os
import ssl

def transcriber(audio_filename):
    try:
        model = whisper.load_model("large")
        transcript = None
    except Exception as e:
        print('no modelo')
 # Let's go with 'base' for a general-purpose model
    try:
        result = model.transcribe(audio_filename)
        transcript = result["text"]
        print("\nTranscription complete! Here's what Jupyternaut heard:\n")
        print("-" * 80) # A nice separator for the transcript
        print(transcript)
        print("-" * 80)
    except Exception as e:
        print(f"Jupyternaut ran into an issue during transcription: {e}")
    return transcript # You can use this return value if you want to do more with the transcript!



In [ ]:
transcriber('audio.mp4')

 25%|█████████▍                            | 728M/2.88G [00:21<00:45, 50.9MiB/s]